# CODIGO DE RE-ENTRENAMIENTO

*NOTA: Para poder usar este código de entrenamiento hay que lanzarlo desde exactamente el mismo entorno en el que fue creado.*

*Se puede instalar ese entorno en la nueva máquina usando el environment.yml que creamos en el set up del proyecto*

*Copiar el proyecto1.yml al directorio y en el terminal o anaconda prompt ejecutar:*

conda env create --file proyecto1.yml --name proyecto1

In [1]:
### ----- IMPORTACIÓN DE LIBRERÍAS ----- ###
import numpy as np
import pandas as pd
import cloudpickle

from janitor import clean_names

from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

### ----- CARGA DE DATOS ----- ###
ruta_proyecto = 'C:/Users/Alfonso/OneDrive/Documentos_/Formación/Bootcamp_DS/Repo_DS/Proyect_Break_ML/Proyecto_ML/00_PROYECTO_ML'

nombre_fichero_datos = 'marketing_campaign.xlsx'

ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_excel(ruta_completa,index_col=0)


### ----- LIMPIEZA DE DATOS ----- ###
df = (clean_names(df) \
             .drop_duplicates())

### ----- CREACIÓN DE NUEVAS VARIABLES ----- ###
# Miembros totales en casa
dicc_status = {
    'Together': 2,  # Generalmente representa a dos personas viviendo juntas
    'Divorced': 1,  # Generalmente representa a una persona que está divorciada
    'Married': 2,   # Generalmente representa a dos personas casadas
    'Single': 1,    # Generalmente representa a una persona soltera
    'Widow':1,      # Generalmente representa a una persona viuda
    'OTROS': 1      # 1 por defecto    
}

df['status_members'] = df['marital_status'].map(dicc_status)
df['household_members'] = df['status_members']+df['kidhome']+df['teenhome']

# Nº ofertas aceptadas
df['total_cmp'] = df.filter(like='accepted').apply(pd.to_numeric, errors='coerce').sum(axis=1)

# Elimino columnas intermedias 
df = df.drop(columns=['status_members'])

# ----- INSTANCIO VARIABLES FINALES ----- #                  
variables_finales = ['household_members',
                     'income',
                     'mntgoldprods',
                     'mntmeatproducts',
                     'mntwines',
                     'numcatalogpurchases',
                     'numstorepurchases',
                     'numwebvisitsmonth',
                     'total_cmp' 
                  ]

### ----- INSTANCIO VARIABLE TARGET ----- ###
target = 'response'

### ----- DIVISIÓN X,Y ----- ### 
x = df[variables_finales].copy()

y = df[target].copy()


### ----- CARGA DEL PIPE ----- ### 
nombre_pipe_entrenamiento = 'pipe_entrenamiento.pickle'

ruta_pipe_entrenamiento = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento

with open(ruta_pipe_entrenamiento, mode='rb') as file:
   pipe_entrenamiento = cloudpickle.load(file)

### ----- ENTRENAMIENTO ----- ###
pipe_ejecucion = pipe_entrenamiento.fit(x,y)


### ----- GUARDADO DEL MODELO FINAL ----- ###
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)